In [1]:
# import os
# import shutil

# def clear_directory(dir_path):
#     for filename in os.listdir(dir_path):
#         file_path = os.path.join(dir_path, filename)
#         if os.path.isfile(file_path) or os.path.islink(file_path):
#             os.unlink(file_path)  # remove file or symlink
#         elif os.path.isdir(file_path):
#             shutil.rmtree(file_path)  # remove directory

# # Usage:
# dir_path = "/kaggle/working/"
# clear_directory(dir_path)

In [2]:
# %pip install --upgrade tensorflow keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score
import pandas as pd
import shutil
import os
import numpy as np

# example of model file names : 
# model_imagesPerEntity_1647_imgSize_16_dataPercentage_100_batchSize_32_epochs_100.keras
# model_imagesPerEntity_2481_imgSize_64_dataPercentage_100_batchSize_32_epochs_100.keras

def copy_models_to_kaggle(model_paths):
    os.makedirs('/kaggle/working/models', exist_ok=True)
    new_paths = []
    for model_path in model_paths:
        model_file_name = model_path.split("/")[-1]        
        new_model_path = '/kaggle/working/models/' + model_file_name
        new_paths.append(new_model_path)
        if os.path.exists(new_model_path):
            print(f"model already exists at {new_model_path}")
        shutil.copy(model_path, new_model_path)
    return new_paths
 

batch_size = 32
test_dataset_paths = [
    r"/kaggle/input/arabic-letters-900-images-per-letter",
    r"/kaggle/input/arabic-letters-1000-images-per-letter",
    r"/kaggle/input/clean-2500-letters-only"
]
models_paths = [
        r"/kaggle/input/900-model/keras/no-augmentaton/1/model_imagesPerEntity_918_imgSize_16_dataPercentage_100_batchSize_32_epochs_100.keras",
        r"/kaggle/input/900-model/keras/no-augmentaton/1/model_imagesPerEntity_918_imgSize_32_dataPercentage_100_batchSize_32_epochs_100.keras",
        r"/kaggle/input/900-model/keras/no-augmentaton/1/model_imagesPerEntity_918_imgSize_64_dataPercentage_100_batchSize_32_epochs_100.keras",
        r"/kaggle/input/900-model/keras/no-augmentaton/1/model_imagesPerEntity_918_imgSize_128_dataPercentage_100_batchSize_32_epochs_100.keras",
        r"/kaggle/input/900-model/keras/no-augmentaton/1/model_imagesPerEntity_918_imgSize_256_dataPercentage_100_batchSize_32_epochs_100.keras",
        r"/kaggle/input/900-model/keras/no-augmentaton/1/model_imagesPerEntity_918_imgSize_300_dataPercentage_100_batchSize_32_epochs_100.keras",

        r"/kaggle/input/1000-model/keras/no-augmentation/1/model_imagesPerEntity_1048_imgSize_16_dataPercentage_100_batchSize_32_epochs_100.keras",
        r"/kaggle/input/1000-model/keras/no-augmentation/1/model_imagesPerEntity_1048_imgSize_32_dataPercentage_100_batchSize_32_epochs_100.keras",
        r"/kaggle/input/1000-model/keras/no-augmentation/1/model_imagesPerEntity_1048_imgSize_64_dataPercentage_100_batchSize_32_epochs_100.keras",
        r"/kaggle/input/1000-model/keras/no-augmentation/1/model_imagesPerEntity_1048_imgSize_128_dataPercentage_100_batchSize_32_epochs_100.keras",
        r"/kaggle/input/1000-model/keras/no-augmentation/1/model_imagesPerEntity_1048_imgSize_256_dataPercentage_100_batchSize_32_epochs_100.keras",
        r"/kaggle/input/1000-model/keras/no-augmentation/1/model_imagesPerEntity_1048_imgSize_300_dataPercentage_100_batchSize_32_epochs_100.keras",

        r"/kaggle/input/2500-model/keras/no-augmentation/1/model_imagesPerEntity_2481_imgSize_16_dataPercentage_100_batchSize_32_epochs_100.keras",
        r"/kaggle/input/2500-model/keras/no-augmentation/1/model_imagesPerEntity_2481_imgSize_32_dataPercentage_100_batchSize_32_epochs_100.keras",
        r"/kaggle/input/2500-model/keras/no-augmentation/1/model_imagesPerEntity_2481_imgSize_64_dataPercentage_100_batchSize_32_epochs_100.keras",
        r"/kaggle/input/2500-model/keras/no-augmentation/1/model_imagesPerEntity_2481_imgSize_128_dataPercentage_100_batchSize_32_epochs_100.keras",
        r"/kaggle/input/2500-model/keras/no-augmentation/1/model_imagesPerEntity_2481_imgSize_256_dataPercentage_100_batchSize_32_epochs_100.keras",
        r"/kaggle/input/2500-model/keras/no-augmentation/1/model_imagesPerEntity_2481_imgSize_300_dataPercentage_100_batchSize_32_epochs_100.keras"
]

print("copying models to kaggle...")
copied_models_paths = copy_models_to_kaggle(models_paths)
print(f"loading models in RAM...")
model_path_to_model = {}
for model_path in copied_models_paths:
    print(f"loading model in : {model_path} ...")
    model = load_model(model_path)
    model_path_to_model[model_path] = model

all_datasets_accuracies = {}
for dataset_path in test_dataset_paths:
    dataset_name  = dataset_path.split("/")[-1]
    print(f"Testing dataset : {dataset_path} ...")
    dataset_test_accuracies = {}
    for model_path in copied_models_paths:
        model_file_name_parts = model_path.split("/")[-1].split(".")[0].split("_")
        trained_data_size = model_file_name_parts[2]
        img_size = model_file_name_parts[4]
        current_model_name =  f"model_imagesPerEntity_{trained_data_size}_imgSize_{img_size}"
        model = model_path_to_model[model_path]
        print(f"trained_data_size : {trained_data_size}, img_size : {img_size}, model_name : {current_model_name} ...")
        test_datagen = ImageDataGenerator(rescale=1./255)
        test_generator = test_datagen.flow_from_directory(
            dataset_path,
            target_size=(int(img_size), int(img_size)),
            batch_size=batch_size,
            class_mode='categorical',
            color_mode='grayscale',
            shuffle=False)
        print()

        # Predict the output
        steps = int(np.ceil(test_generator.samples / test_generator.batch_size))
        predictions = model.predict(test_generator, steps=steps)
        predictions = predictions.argmax(axis=-1)                    
        true_classes = test_generator.classes
        accuracy =  round(accuracy_score(true_classes, predictions), 3)
        dataset_test_accuracies[current_model_name] = accuracy
        print(f"Accuracy of model {current_model_name} on dataset {dataset_name} : {accuracy}")
    all_datasets_accuracies[dataset_name] = dataset_test_accuracies

print(f"all datasets accuracies : {all_datasets_accuracies}")
accuracies_df = pd.DataFrame(all_datasets_accuracies).transpose()
accuracies_df = accuracies_df.style.background_gradient(cmap='tab20_r').set_table_styles([dict(selector="th", props=[("text-align", "center")])])
display(accuracies_df)


2024-05-05 18:29:22.717789: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-05 18:29:22.717923: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-05 18:29:22.852253: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


copying models to kaggle...
loading models in RAM...
loading model in : /kaggle/working/models/model_imagesPerEntity_918_imgSize_16_dataPercentage_100_batchSize_32_epochs_100.keras ...
loading model in : /kaggle/working/models/model_imagesPerEntity_918_imgSize_32_dataPercentage_100_batchSize_32_epochs_100.keras ...
loading model in : /kaggle/working/models/model_imagesPerEntity_918_imgSize_64_dataPercentage_100_batchSize_32_epochs_100.keras ...
loading model in : /kaggle/working/models/model_imagesPerEntity_918_imgSize_128_dataPercentage_100_batchSize_32_epochs_100.keras ...
loading model in : /kaggle/working/models/model_imagesPerEntity_918_imgSize_256_dataPercentage_100_batchSize_32_epochs_100.keras ...
loading model in : /kaggle/working/models/model_imagesPerEntity_918_imgSize_300_dataPercentage_100_batchSize_32_epochs_100.keras ...
loading model in : /kaggle/working/models/model_imagesPerEntity_1048_imgSize_16_dataPercentage_100_batchSize_32_epochs_100.keras ...
loading model in : 

/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


804/804 ━━━━━━━━━━━━━━━━━━━━ 118s 146ms/step
Accuracy of model model_imagesPerEntity_918_imgSize_16 on dataset arabic-letters-900-images-per-letter : 0.273
trained_data_size : 918, img_size : 32, model_name : model_imagesPerEntity_918_imgSize_32 ...
Found 25704 images belonging to 28 classes.



/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


...

/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2171/2171 ━━━━━━━━━━━━━━━━━━━━ 69s 32ms/step
Accuracy of model model_imagesPerEntity_2481_imgSize_256 on dataset clean-2500-letters-only : 0.998
trained_data_size : 2481, img_size : 300, model_name : model_imagesPerEntity_2481_imgSize_300 ...
Found 69468 images belonging to 28 classes.

   5/2171 ━━━━━━━━━━━━━━━━━━━━ 1:19 37ms/step

/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2171/2171 ━━━━━━━━━━━━━━━━━━━━ 77s 36ms/step
Accuracy of model model_imagesPerEntity_2481_imgSize_300 on dataset clean-2500-letters-only : 0.979
all datasets accuracies : {'arabic-letters-900-images-per-letter': {'model_imagesPerEntity_918_imgSize_16': 0.273, 'model_imagesPerEntity_918_imgSize_32': 0.681, 'model_imagesPerEntity_918_imgSize_64': 0.904, 'model_imagesPerEntity_918_imgSize_128': 0.719, 'model_imagesPerEntity_918_imgSize_256': 0.586, 'model_imagesPerEntity_918_imgSize_300': 0.09, 'model_imagesPerEntity_1048_imgSize_16': 0.216, 'model_imagesPerEntity_1048_imgSize_32': 0.477, 'model_imagesPerEntity_1048_imgSize_64': 0.641, 'model_imagesPerEntity_1048_imgSize_128': 0.474, 'model_imagesPerEntity_1048_imgSize_256': 0.223, 'model_imagesPerEntity_1048_imgSize_300': 0.471, 'model_imagesPerEntity_2481_imgSize_16': 0.076, 'model_imagesPerEntity_2481_imgSize_32': 0.104, 'model_imagesPerEntity_2481_imgSize_64': 0.079, 'model_imagesPerEntity_2481_imgSize_128': 0.104, 'model_imagesPerEnt

,model_imagesPerEntity_918_imgSize_16,model_imagesPerEntity_918_imgSize_32,model_imagesPerEntity_918_imgSize_64,model_imagesPerEntity_918_imgSize_128,model_imagesPerEntity_918_imgSize_256,model_imagesPerEntity_918_imgSize_300,model_imagesPerEntity_1048_imgSize_16,model_imagesPerEntity_1048_imgSize_32,model_imagesPerEntity_1048_imgSize_64,model_imagesPerEntity_1048_imgSize_128,model_imagesPerEntity_1048_imgSize_256,model_imagesPerEntity_1048_imgSize_300,model_imagesPerEntity_2481_imgSize_16,model_imagesPerEntity_2481_imgSize_32,model_imagesPerEntity_2481_imgSize_64,model_imagesPerEntity_2481_imgSize_128,model_imagesPerEntity_2481_imgSize_256,model_imagesPerEntity_2481_imgSize_300
arabic-letters-900-images-per-letter,0.273000,0.681000,0.904000,0.719000,0.586000,0.090000,0.216000,0.477000,0.641000,0.474000,0.223000,0.471000,0.076000,0.104000,0.079000,0.104000,0.091000,0.077000
arabic-letters-1000-images-per-letter,0.255000,0.428000,0.496000,0.259000,0.180000,0.088000,0.698000,0.782000,0.933000,0.802000,0.555000,0.898000,0.169000,0.183000,0.199000,0.171000,0.137000,0.093000
clean-2500-letters-only,0.178000,0.274000,0.222000,0.083000,0.068000,0.071000,0.279000,0.327000,0.388000,0.234000,0.064000,0.122000,0.975000,0.981000,0.999000,0.999000,0.998000,0.979000
